## Import Dependencies and Tools
This section loads necessary libraries and adds the utils folder to the import path.

In [1]:

import sys
sys.path.append('../')
import os

from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from utils.whisper import whisper_tool
from agents.langchain_agent import agent_with_memory


/Users/adriannadziadyk/Multimodal-AI-YouTube-QA-Bot/notebooks/../agents/langchain_agent.py:27: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.load_local(vectorstore_path, OpenAIEmbeddings(), allow_dangerous_deserialization=True)
/Users/adriannadziadyk/Multimodal-AI-YouTube-QA-Bot/notebooks/../agents/langchain_agent.py:51: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm

## Load Vectorstore and Build Retriever
Load the FAISS vector store to enable transcript-based question answering.

In [2]:
# Use current working directory since __file__ is not defined in notebooks
project_root = os.getcwd()
vectorstore_path = os.path.join(project_root, "../data/vectorstores/eleo_faiss")

db = FAISS.load_local(vectorstore_path, OpenAIEmbeddings(), allow_dangerous_deserialization=True)

retriever = db.as_retriever()

In [3]:
#test
print("Looking for FAISS files in:", os.path.abspath(vectorstore_path))
print("Found files:", os.listdir(vectorstore_path))


Looking for FAISS files in: /Users/adriannadziadyk/Multimodal-AI-YouTube-QA-Bot/data/vectorstores/eleo_faiss
Found files: ['index.faiss', 'index.pkl']


## Define Custom Prompt Template
Create a custom prompt to ensure the agent only answers using transcript content.

In [4]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate.from_template(
    template="""
You are an assistant that answers questions only using the following transcript chunk.
If the answer is not contained within it, reply with "I don't know."

Transcript:
{context}

Question: {question}
Answer:
"""
)


## Create Retrieval QA Chain
Build a chain that uses the retriever and custom prompt for context-aware answers.

In [5]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=False
)

def answer_question_only_result(query: str) -> str:
    return qa_chain.invoke({"query": query})["result"]

qa_tool = Tool(
    name="VideoQA",
    func=answer_question_only_result,
    description="Answers questions about the Eleo German learning video"
)


## Initialize Tools and LLM
Combine QA and Whisper tools to equip the agent for multimodal tasks.

In [6]:
tools = [qa_tool, whisper_tool]
llm = ChatOpenAI(model="gpt-3.5-turbo")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


## 🧪 LangSmith Tracing Setup
Define a tracked function for LangSmith monitoring.

In [7]:
from langsmith import traceable

@traceable(name="Eleo_QA_Session")
def ask_agent_traced(question: str):
    return agent_with_memory.run(question)


## 📊 Run Test Questions
Example usage to validate that the system answers accurately.

In [8]:
response_1 = ask_agent_traced("How does Eleo recommend practicing German pronunciation?")
print(response_1)

response_2 = ask_agent_traced("Can you explain that again more simply?")
print(response_2)


/var/folders/m8/k1bt27ks3h3cd9920twfctb40000gn/T/ipykernel_83249/3644087192.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return agent_with_memory.run(question)




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: VideoQA
Action Input: How does Eleo recommend practicing German pronunciation?
Observation: Eleo recommends practicing German pronunciation by reading and listening to materials at B1 or B2 level to improve vocabulary.
Thought:Do I need to use a tool? No
AI: Eleo recommends practicing German pronunciation by reading and listening to materials at B1 or B2 level to improve vocabulary.

> Finished chain.
Eleo recommends practicing German pronunciation by reading and listening to materials at B1 or B2 level to improve vocabulary.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Eleo recommends practicing German pronunciation by reading and listening to materials at a B1 or B2 level to improve your vocabulary. This can help you become more comfortable with German sounds and pronunciation. Practice regularly to see improvement.

> Finished chain.
Eleo recommends practicing Ge

###  LangChain RetrievalQA setup
        

In [9]:
#Agent initialisation code
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

In [10]:
response = agent_executor.run("What does 'Guten Tag' mean?")
print(response)




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: "Guten Tag" is a German greeting that means "Good day" or "Good afternoon." It is a common way to say hello in German.

> Finished chain.
"Guten Tag" is a German greeting that means "Good day" or "Good afternoon." It is a common way to say hello in German.
